<a href="https://colab.research.google.com/github/sime1/notebooks/blob/master/sklearn_feature_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature selection in sklearn

In questo notebook provo ad utilizzare i diversi metodi di feature selection offerti da sklearn. Per applicare i metodi utilizzo il dataset `covertype` fornito da `sklearn`.


In [0]:
import numpy as np
from sklearn.datasets import fetch_covtype

ds = fetch_covtype()
X, y = ds.data, ds.target
print(ds.DESCR)

.. _covtype_dataset:

Forest covertypes
-----------------

The samples in this dataset correspond to 30×30m patches of forest in the US,
collected for the task of predicting each patch's cover type,
i.e. the dominant species of tree.
There are seven covertypes, making this a multiclass classification problem.
Each sample has 54 features, described on the
`dataset's homepage <https://archive.ics.uci.edu/ml/datasets/Covertype>`__.
Some of the features are boolean indicators,
while others are discrete or continuous measurements.

**Data Set Characteristics:**

    =================   ============
    Classes                        7
    Samples total             581012
    Dimensionality                54
    Features                     int
    =================   ============

:func:`sklearn.datasets.fetch_covtype` will load the covertype dataset;
it returns a dictionary-like object
with the feature matrix in the ``data`` member
and the target values in ``target``.
The dataset will be d

## Panoramica

Il processo di feature selection si occupa di ridurre il numero di variabili del dataset che il modello prende in considerazione, con lo scopo di migliorare l'accuracy del modello evitando l'overfitting

### `VarianceThreshold`

`VarianceThreshold` è un *feature selector* che rimuove le features che non raggiungono una determinata varianza. Con i parametri di default, questo si traduce nella rimozione delle feature che hanno lo stesso valore per tutti i sample (varianza 0)

In [0]:
from sklearn.feature_selection import  VarianceThreshold

sel = VarianceThreshold(threshold=0.01)
sel.fit(X)
filtered = sel.transform(X)

print(X.shape)
print(filtered.shape)

(581012, 54)
(581012, 33)


Come si può vedere, con questo metodo utilizzando un threshold di `0.01` il numero di feature viene ridotto da 54 a 33

### `SelectKBest` e `SelectPercentile`

Qesti due feature selector permettono di selezionare le feature che hanno score più alto; `SelectKBest` seleziona un numero prefissato di feature, mentre `SelectPercentile` seleziona una percentuale delle feature.

Entrambe le funzioni chiedono di specificare la funzione da utilizzare per calcolare lo score delle feature

In [0]:
from sklearn.feature_selection import SelectKBest, SelectPercentile, mutual_info_classif

sel_kbest = SelectKBest(score_func=mutual_info_classif,k=30)
sel_percent = SelectPercentile(score_func=mutual_info_classif, percentile=60)

print(X.shape)

sel_kbest.fit(X[:1000], y[:1000])
kbest_data = sel_kbest.transform(X)
print(kbest_data.shape)

sel_percent.fit(X[:1000], y[:1000])
percent_data = sel_percent.transform(X)
print(percent_data.shape)

(581012, 54)
(581012, 30)
(581012, 32)


In questo caso ho utilizzato un numero ridotto di sample, in quanto utilizzando l'intero dataset impiega molto tempo. Da notare comunque che questi metodi andrebbero applicati solo al set di training e non all'intero dataset.

### `RFE` e `RFECV`

Viene utilizzato un predittore esterno che assegna pesi alle diverse feature. Viene effettuato il training; i pesi ottenuti durante il training vengono considerati come indicatore dell'importanza delle features, quindi viene effettuato il pruning delle features meno importanti. Questa procedura è ripetuta più volte, finchè non si raggiunge il numero desiderato di features.

Nel caso di `REFCV` questo processo viene effettuato iterativamente, compiendo cross validation, per determinare il numero ottimale di features

In [0]:
from sklearn.feature_selection import RFE, RFECV
from sklearn.linear_model import Lasso

est = Lasso()

sel_rfe = RFE(estimator=est, n_features_to_select=30)
sel_rfecv = RFECV(estimator=est)

print(X.shape)

sel_rfe.fit(X[:10000], y[:10000])
rfe_data = sel_rfe.transform(X)
print(rfe_data.shape)

sel_percent.fit(X[:10000], y[:10000])
percent_data = sel_percent.transform(X)
print(percent_data.shape)

(581012, 54)
(581012, 30)
(581012, 32)


Anche in questo caso ho utilizzato un numero ridotto di sample

### `SelectFromModel`

Anche questo metodo funziona utilizzando un predittore esterno e valutando l'importanza delle feature; in questo caso però invece di utilizzaro un procedimento ricorsivo, vengono utilizzati un valore di threshold oppure delle euristiche per la selezione delle features ritenute importanti.

Le euristiche messe a disposizione da `sklearn` sono `mean`, `median` e loro multipli

In [0]:
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier

est = DecisionTreeClassifier()
sel = SelectFromModel(estimator=est, threshold='0.5*mean')

print(X.shape)

sel.fit(X, y)
filtered = sel.transform(X)
print(filtered.shape)

(581012, 54)
(581012, 15)


In questo codice ho utilizzato un albero di decisione come classificatore, ed ho utilizzato il valore `'0.5*mean'` come threshold (ossia $1/2$ della media); il feature selector ha selezionato le 15 feature più importanti